In [1]:
#データの読み込み
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/train.txt', sep="\t")
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/test.txt', sep="\t")
valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/valid.txt', sep="\t")
# データ数の確認
print('学習データ')
print(train['CATEGORY'].value_counts())
print('検証データ')
print(valid['CATEGORY'].value_counts())
print('評価データ')
print(test['CATEGORY'].value_counts())

学習データ
CATEGORY
b    4502
e    4223
t    1219
m     728
Name: count, dtype: int64
検証データ
CATEGORY
b    562
e    528
t    153
m     91
Name: count, dtype: int64
評価データ
CATEGORY
b    563
e    528
t    152
m     91
Name: count, dtype: int64


In [2]:
# 単語の辞書を作成
from collections import Counter
words = []
for text in train['TITLE']:                #訓練データから文章を1つずつ取り出す
    for word in text.rstrip().split():     #文章を単語に分解
        words.append(word)                 #単語をリストに追加
c = Counter(words)                         #単語の出現回数を数える
print(c.most_common(10))                   #頻度上位10単語
word2id = {}                               #単語IDの辞書
for i, cnt in enumerate(c.most_common()):  #頻度上位10単語分繰り返す
    if cnt[1] > 1:                         #出現回数が1より大きい単語のみ
        word2id[cnt[0]] = i + 1            #辞書に単語とIDを紐付ける
for i, cnt in enumerate(word2id.items()):  #辞書の中身を確認
    if i >= 10:                            #10単語だけ表示
        break                              #for文を抜ける
    print(cnt[0], cnt[1])                  #単語とIDを表示

[('to', 2151), ('...', 2031), ('in', 1415), ('as', 1027), ('on', 1025), ('UPDATE', 1000), ('-', 991), ('for', 969), ('of', 957), ('The', 859)]
to 1
... 2
in 3
as 4
on 5
UPDATE 6
- 7
for 8
of 9
The 10


In [3]:
# 単語のID化
def tokenizer(text):                                 #単語IDのリストを返す関数
    words = text.rstrip().split()                    #単語に分解
    return [word2id.get(word, 0) for word in words]  #単語のIDに変換

sample = train.at[0, 'TITLE']                        #学習データの1つ目の文章
print(sample)                                        #文章を表示
print(tokenizer(sample))                             #文章を単語IDに変換

Justin Bieber Under Investigation For Attempted Robbery At Dave & Buster's
[66, 79, 733, 2094, 21, 4933, 6674, 35, 1514, 86, 0]


In [7]:
# RNNの作成
# モデルの構築
import random
import torch
from torch import nn
import torch.utils.data as data
from torchinfo import summary
import numpy as np

# 乱数のシードを設定
# parserなどで指定
seed = 1234

random.seed(seed)                                   # Python標準ライブラリの乱数のシードを設定
np.random.seed(seed)                                # Numpy乱数のシードを設定
torch.manual_seed(seed)                             # PyTorch乱数のシードを設定
torch.cuda.manual_seed(seed)                        # PyTorchのCUDA乱数のシードを設定
torch.backends.cudnn.benchmark = False              # PyTorchのCUDNNのベンチマークを使用しない  (cudnn内の非決定的な処理の固定化)
torch.backends.cudnn.deterministic = True           # PyTorchのCUDNNの定着を使用

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32      # 乱数生成のシードの初期値を設定
    np.random.seed(worker_seed)                     # Numpy乱数のシードを設定
    random.seed(worker_seed)                        # Python標準ライブラリの乱数のシードを設定

g = torch.Generator()                               # PyTorch乱数のシードを設定
g.manual_seed(seed)                                 # 乱数生成器にシードを設定

class RNN(nn.Module):                                                                               # RNNクラスを定義
    def __init__(self, vocab_size, emb_size, padding_idx, hidden_size, output_size, num_layers=1):  # コンストラクタ
        super().__init__()                                                                          # 親クラスのコンストラクタを呼ぶ
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)                      # 単語埋め込み層
        self.rnn = nn.LSTM(emb_size, hidden_size, batch_first=True)                                 # RNN層
        self.fc = nn.Linear(hidden_size, output_size)                                               # 全結合層

    def forward(self, x, h0=None):                                                                  # 順伝播処理
        x = self.emb(x)                                                                             # 単語埋め込み
        x, h = self.rnn(x, h0)                                                                      # RNN
        x = x[:, -1, :]                                                                             # 最後のステップのみを抽出
        logits = self.fc(x)                                                                         # 全結合層
        return logits                                                                               # 出力

# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 2  # 辞書のID数 + unknown + パディングID
EMB_SIZE = 300                               # 単語埋め込みベクトルのサイズ
PADDING_IDX = len(set(word2id.values())) + 1 # パディングID
OUTPUT_SIZE = 4                              # 出力サイズ
HIDDEN_SIZE = 50                             # 隠れ層のサイズ
NUM_LAYERS = 1                               # RNN層の数

# モデルの定義
model = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)  # RNNクラスのインスタンスを作成
print(model)                                                                          # モデルの構造を確認

RNN(
  (emb): Embedding(10328, 300, padding_idx=10327)
  (rnn): LSTM(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)


In [8]:
x = torch.tensor([tokenizer(sample)], dtype=torch.int64)                           # 文章を単語IDに変換
print(x)                                                                           # 文章をIDでを表示
print(x.size())                                                                    # 文章のサイズを確認
print(nn.Softmax(dim=-1)(model(x)))                                                # 出力を確認

tensor([[  66,   79,  733, 2094,   21, 4933, 6674,   35, 1514,   86,    0]])
torch.Size([1, 11])
tensor([[0.2148, 0.2503, 0.2325, 0.3023]], grad_fn=<SoftmaxBackward0>)


In [9]:
# ターゲットのテンソル化
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
Y_train = torch.from_numpy(train['CATEGORY'].map(category_dict).values)
Y_valid = torch.from_numpy(valid['CATEGORY'].map(category_dict).values)
Y_test = torch.from_numpy(test['CATEGORY'].map(category_dict).values)
print(Y_train.size())
print(Y_train)

torch.Size([10672])
tensor([2, 0, 2,  ..., 0, 0, 0])


In [10]:
class NewsDataset(data.Dataset):
    """
    newsのDatasetクラス

    Attributes
    ----------------------------
    X : データフレーム
        単語ベクトルの平均をまとめたテンソル
    y : テンソル
        カテゴリをラベル化したテンソル
    phase : 'train' or 'val'
        学習か訓練かを設定する
    """
    def __init__(self, X, y, phase='train'):
        self.X = X['TITLE']
        self.y = y
        self.phase = phase

    def __len__(self):
        """全データサイズを返す"""
        return len(self.y)

    def __getitem__(self, idx):
        """idxに対応するテンソル形式のデータとラベルを取得"""
        inputs = torch.tensor(tokenizer(self.X[idx]))
        return inputs, self.y[idx]

train_dataset = NewsDataset(train, Y_train, phase='train')
valid_dataset = NewsDataset(valid, Y_valid, phase='val')
test_dataset = NewsDataset(test, Y_test, phase='val')
# 動作確認
idx = 0
print(train_dataset.__getitem__(idx)[0].size())
print(train_dataset.__getitem__(idx)[1])
print(valid_dataset.__getitem__(idx)[0].size())
print(valid_dataset.__getitem__(idx)[1])
print(test_dataset.__getitem__(idx)[0].size())
print(test_dataset.__getitem__(idx)[1])

torch.Size([11])
tensor(2)
torch.Size([11])
tensor(3)
torch.Size([13])
tensor(2)


In [11]:
# DataLoaderを作成
batch_size = 1

train_dataloader = data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
valid_dataloader = data.DataLoader(
            valid_dataset, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_dataloader = data.DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g)

dataloaders_dict = {'train': train_dataloader,
                    'val': valid_dataloader,
                    'test': test_dataloader,
                   }

# 動作確認
batch_iter = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iter)
print(inputs.size())
print(labels)

torch.Size([1, 12])
tensor([2])


In [12]:
import matplotlib.pyplot as plt
from tqdm import tqdm

# 学習用の関数を定義
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []
    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {} / {}'.format(epoch + 1, num_epochs))
        print('--------------------------------------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 訓練モード
            else:
                net.eval() # 検証モード

            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                optimizer.zero_grad() # optimizerを初期化

                # 順伝播計算(forward)
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) # 損失を計算
                    _, preds = torch.max(outputs, 1) # ラベルを予想

                    # 訓練時は逆伝播
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イテレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率の表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)

            print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    return train_loss, train_acc, valid_loss, valid_acc

# 学習を実行する

# モデルの定義
net = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
net.train()

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 最適化手法の定義
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

num_epochs = 5
train_loss, train_acc, valid_loss, valid_acc = train_model(net,
            dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1 / 5
--------------------------------------------


100%|██████████| 10672/10672 [04:45<00:00, 37.42it/s]


train Loss: 0.9923, Acc: 0.6209


100%|██████████| 1334/1334 [00:01<00:00, 1234.35it/s]


val Loss: 0.8070, Acc: 0.7069
Epoch 2 / 5
--------------------------------------------


...

val Loss: 0.6004, Acc: 0.7826
Epoch 5 / 5
--------------------------------------------


100%|██████████| 10672/10672 [06:10<00:00, 28.80it/s]


train Loss: 0.1649, Acc: 0.9457


100%|██████████| 1334/1334 [00:01<00:00, 920.68it/s]

val Loss: 0.5966, Acc: 0.8028


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install torchinfo